## 어떤 무선 청소기가 인기 있을까?

### 한페이지 검색해보기

In [ ]:
# selenium으로 다나와 검색 결과 URL에 접속
from selenium import webdriver
driver = webdriver.Chrome('D:\playwithdata\chromedriver.exe')
url = 'http://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&previousKeyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&volumeType=allvs&page=2&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=103740&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2468&defaultVaTab=141157&tab=goods'
driver.get(url)

In [ ]:
from bs4 import BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 페이지에 대한 무선청소기 정보 가져오기
prod_items = soup.select('li.prod_item')
len(prod_items)

In [ ]:
# 페이지에 대한 무선청소기 정보 가져오기
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
# 페이지에 대한 무선청소기 정보 가져오기
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
len(prod_items)

In [ ]:
# 상품명 정보 가져오기
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

In [ ]:
# 스펙 목록 정보 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
print(spec_list)

In [ ]:
# 가격 정보 가져오기
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

In [ ]:
prod_data = []
for prod_item in prod_items:
    try: # 상품명 가져오기
        title = prod_item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try: # 스펙 목록 가져오기
        spec_list = prod_item.select('div.spec_list')[0].text.strip()
    except:
        spec_list = ''
    try: # 가격 정보 가져오기 
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
    except:
        price = 0
    prod_data.append([title, spec_list, price])
print(len(prod_data))
print(prod_data)

In [ ]:
# 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
         # 상품명 가져오기
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
        # 스펙 목록 가져오기
        try: 
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
        except:
            spec_list = ''
        # 가격 정보 가져오기 
        try: 
            price = int(prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",",""))
        except:
            price = 0
        prod_data.append([title, spec_list, price])
    return prod_data

In [ ]:
# 상품 정보를 가져오기
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

In [ ]:
print(len(prod_data))
print(prod_data)

### 여러 페이지에 걸친 다나와 검색 페이지 크롤링 하기

In [ ]:
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

In [ ]:
# 쥬피터에 진행표시줄을 위한 tqdm 라이브러리 설치 
! pip install tqdm

In [ ]:
import time
from tqdm import tqdm_notebook
total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. 
    time.sleep(5)

In [ ]:
# 크롤링하기
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
driver = webdriver.Chrome('D:\playwithdata\chromedriver.exe')

# 페이지 로드되는 시간을 위해 3초 기다림
driver.implicitly_wait(3)
keyword = '무선청소기'
total_page = 10
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용, 반복문 하나 실행하는것을 하나의 iteration으로 판단
for page in tqdm_notebook(range(1, total_page + 1)):
    # 검색 페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    # 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)
    
    # 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 상품 정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
    prod_item_list = get_prod_items(prod_items)
    # 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list

In [ ]:
prod_data_total

In [ ]:
# 수집된 데이터 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('./danawa/danawa_crawling_result.xlsx', index = False)

### 데이터 전처리

In [ ]:
#크롤링 결과 가져오기
data = pd.read_excel('./danawa/danawa_crawling_result.xlsx')
data.info()
data.head()

In [ ]:
# 회사명과 모델명의 분리
company_list = []
product_list = []
for title in data['상품명']:
    title_info = title.split(' ', 1)    
    if len(title_info) > 1:
        company_name = title_info[0]
        product_name = title_info[1]
        company_list.append(company_name)
        product_list.append(product_name)
    else:
        company_name = title_info[0]
        product_name = None
        company_list.append(company_name)
        product_list.append(product_name)

In [ ]:
# 상품명 확인
data['상품명'][:10]

In [ ]:
# 첫번째 공백 기준으로 분리
title = "LG전자 오브제컬렉션 코드제로 ThinQ A9S AO9571"
info = title.split(' ',1)
print(info)

In [ ]:
print(len(data))
print(len(company_list))
print(company_list[:5])
print(len(product_list))
print(product_list[:5])

In [ ]:
# 카테고리, 사용시간, 흡입력 추출
data['스펙 목록'][0]

In [ ]:
#앞뒤 공백있는 슬래시로 구분, split()함수를 이용하여 분리
spec_list = data['스펙 목록'][0].split(' / ')
spec_list 

In [ ]:
category = spec_list[0] 
category 

In [ ]:
#사용시간, 흡입력이 포함된 원소 추출
for spec in spec_list:
    if '사용시간' in spec: 
         use_time_spec = spec 
    elif '흡입력' in spec:
        suction_spec = spec  
print(use_time_spec)
print(suction_spec)

In [ ]:
# 정량적인 수치 추출
use_time_value = use_time_spec.split(' ')[1].strip() 
suction_value = suction_spec.split(' ')[1].strip()
print(use_time_value)
print(suction_value)

In [ ]:
category_list = []
use_time_list = []
suction_list = [] 
for spec_data in data['스펙 목록']:
    # ' / ' 기준으로 스펙 분리하기 
    spec_list = spec_data.split(' / ')
    
    # 카테고리 추출하기
    category = spec_list[0] 
    category_list.append(category)
        
    # 사용시간, 흡입력 정보가 없는 제품을 위해 변수를 생성 
    use_time_value = None 
    suction_value = None 
    
    # spec_list의 각 원소에서 사용시간, 흡입력 수치 추출
    # split함수로 두번째 원소 추출 후 공백 제거 위해 strip함수 이용
    for spec in spec_list:
        if '사용시간' in spec: 
            use_time_value = spec.split(' ')[1].strip()             
        if '흡입력' in spec:
            suction_value = spec.split(' ')[1].strip() 
            if suction_value == "흡입력:":
                suction_value = spec.split(' ')[2].strip() 
    use_time_list.append(use_time_value)
    suction_list.append(suction_value)

In [ ]:
print("카테고리", len(category_list), category_list[0:5])
print("사용시간", len(use_time_list), use_time_list[0:5])
print("흡입력", len(suction_list), suction_list[26:30])

In [ ]:
use_time_list

In [ ]:
def convert_time_minute(time):
    try:
        if '시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
            else:
                minute = 0
        else: 
            hour = 0
            if '~' in time:
                minute = time.split('~')[0]
            else:
                minute = time.split('분')[0]
        return int(hour)*60 + int(minute)
    except:
        return None                

In [ ]:
times = ["40분", "4분", "1시간", "3시간30분", "4시간", "9~15분"]
for time in times:
    time_value = convert_time_minute(time)
    print(time, "=", time_value)

In [ ]:
# 모델별 사용시간을 분 단위로 통일
new_use_time_list = []
for time in use_time_list:
    value = convert_time_minute(time)
    new_use_time_list.append(value)
new_use_time_list

In [ ]:
suction_list

In [ ]:
# 흡입력 단위 통일 함수
def get_suction(value):
    try:
        value = value.upper()
        if "AW" in value or "W" in value:
            result = value.replace("A", "").replace("W","")
            result = int(result.replace(",",""))
        elif "PA" in value:
            result = value.replace("PA","")
            result = int(result.replace(",",""))/100
        else:
            result = None
        return result
    except:
        return None

In [ ]:
ex_suction_list = ["100W","10AW","10,000Pa","22000Pa"]
for power in ex_suction_list:
    value = get_suction(power)
    print(power, value)

In [ ]:
# 흡입력 단위 통일시키기
new_suction_list = []
for power in suction_list:
    value = get_suction (power)
    new_suction_list.append(value)

In [ ]:
print(len(new_suction_list))
print(new_suction_list[:10])

In [ ]:
# 전처리 데이터 확인
pd_data = pd.DataFrame()
pd_data['카테고리'] = category_list
pd_data['회사명'] = company_list
pd_data['제품'] = product_list
pd_data['가격'] = data['가격']
pd_data['사용시간'] = new_use_time_list
pd_data['흡입력'] = new_suction_list
pd_data.head()

In [ ]:
# 카테고리 분류 기준 및 데이터 개수 점검
pd_data['카테고리'].value_counts() 

In [ ]:
# 카테고리 별 데이터 개수 확인 후 그 수가 가장 많은 핸디/스틱청소기만 추출
pd_data_final = pd_data[pd_data['카테고리'].isin(['핸디/스틱청소기'])]
len(pd_data_final)

In [ ]:
# 추출된 데이터를 엑셀로 저장
pd_data_final.to_excel('./danawa/danawa_data_final.xlsx', index = False)

### 데이터 분석하기

In [ ]:
danawa_data = pd.read_excel('./danawa/danawa_data_final.xlsx')
danawa_data.head()

In [ ]:
# 흡입력 기준
top_list = danawa_data.sort_values(["흡입력"], ascending=False)
top_list.head()

In [ ]:
# 사용시간 기준
top_list = danawa_data.sort_values(["사용시간"], ascending=False)
top_list.head()

In [ ]:
# 흡입력&사용시간 기준
top_list = danawa_data.sort_values(["사용시간","흡입력"], ascending=False)
top_list.head()

In [ ]:
# 가격, 흡입력, 사용시간 평균
price_mean_value = danawa_data['가격'].mean()
suction_mean_value = danawa_data['흡입력'].mean()
use_time_mean_value = danawa_data['사용시간'].mean()
print("가격 평균값", price_mean_value)
print("흡입력 평균값", suction_mean_value)
print("사용시간 평균값", use_time_mean_value)

In [ ]:
# 평균가격 이하, 평균흡입력 이상, 평균 사용시간 이상 제품 검색
condition_data = danawa_data[
    (danawa_data['가격'] <= price_mean_value) &
    (danawa_data['흡입력'] >= suction_mean_value) &
    (danawa_data['사용시간'] >= use_time_mean_value)]
condition_data

### 시각화

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import platform 
# 그래프에서 한글 표기를 위한 글꼴 변경(윈도우, macOS에 대해 처리)
font_path = ''
if platform.system() == 'Windows': 
    font_path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname = font_path).get_name()
    rc('font', family = font_name)
elif platform.system() == 'Darwin':
    font_path = '/Users/$USER/Library/Fonts/AppleGothic.ttf'
    rc('font', family = 'AppleGothic')
else: 
    print('Check your OS system')
    
%matplotlib inline

In [ ]:
# 가격, 사용시간, 흡입력 값이 없는 것은 삭제. 
chart_data = danawa_data.dropna(axis = 0)
len(chart_data)

In [ ]:
# 흡입력, 사용사간 최대, 최소
suction_max_value = chart_data['흡입력'].max()
suction_mean_value = chart_data['흡입력'].mean()
use_time_max_value = chart_data['사용시간'].max()
use_time_mean_value = chart_data['사용시간'].mean()

In [ ]:
# 청소기 성능 시각화
plt.figure(figsize=(20, 10))
plt.title("무선 핸디/스틱청소기 차트")
sns.scatterplot(x = '흡입력', y = '사용시간', size = '가격', hue = chart_data['회사명'], 
             data = chart_data, sizes = (10, 1000),legend=False)
plt.plot([0, suction_max_value], 
          [use_time_mean_value, use_time_mean_value], 
          'r--', 
          lw = 1 ) #흡입력 평균
plt.plot([suction_mean_value, suction_mean_value], 
          [0, use_time_max_value], 
          'r--', 
          lw = 1 ) #사용시간 평균
plt.show()

In [ ]:
# 상위 20개 선택
chart_data_selected = chart_data[:20]
chart_data_selected

In [ ]:
# 흡입력, 사용시간의 최댓값/최솟값 정리
# 흡입력, 사용시간의 최댓값, 최솟값 구하기
suction_max_value = chart_data_selected['흡입력'].max()
suction_mean_value = chart_data_selected['흡입력'].mean()
use_time_max_value = chart_data_selected['사용시간'].max()
use_time_mean_value = chart_data_selected['사용시간'].mean()

# 청소기 성능 시각화
plt.figure(figsize=(20, 10))
plt.title("무선 핸디/스틱청소기 TOP 20")
sns.scatterplot(x = '흡입력', 
                  y = '사용시간', 
                  size = '가격', 
                  hue = chart_data_selected['회사명'], 
                  data = chart_data_selected, sizes = (100, 2000))
plt.plot([60, suction_max_value], 
          [use_time_mean_value, use_time_mean_value], 
          'r--', 
          lw = 1 )
plt.plot([suction_mean_value, suction_mean_value], 
          [20, use_time_max_value], 
          'r--', 
          lw = 1 )
for index, row in chart_data_selected.iterrows():
    x = row['흡입력']
    y = row['사용시간']
    s = row['제품'].split(' ')[0]
    plt.text(x, y, s, size=17)
plt.show()